<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [8]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from k12libs.utils.nb_easy import K12AI_DATASETS_ROOT

In [4]:
mnist_dir = os.path.join(K12AI_DATASETS_ROOT, 'cv', 'mnist')

In [6]:
# mnist_train_dataset = datasets.ImageFolder(os.path.join(mnist_dir, 'imgs'), transform=transforms.ToTensor())

In [ ]:
dataset = datasets.ImageFolder('train',
                 transform=transforms.ToTensor())

In [ ]:
mean = 0.0
for img, _ in dataset:
    #mean += img.sum([1,2])/torch.numel(img[0])
    mean += img.mean([1,2])
mean = mean/len(dataset)
print(mean)

In [ ]:
sumel = 0.0
countel = 0
for img, _ in dataset:
    sumel += img.sum([1, 2])
    countel += torch.numel(img[0])
mean = sumel/countel
print(mean)

In [ ]:
sumel = 0.0
countel = 0
for img, _ in dataset:
    img = (img - mean.unsqueeze(1).unsqueeze(1))**2
    sumel += img.sum([1, 2])
    countel += torch.numel(img[0])
std = torch.sqrt(sumel/countel)

In [42]:
gdata = torch.randn(1000, 3, 24, 24)

In [46]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = gdata
        
    def __getitem__(self, index):
        x = self.data[index]
        return x

    def __len__(self):
        return len(self.data)
    

dataset = MyDataset()
loader = DataLoader(
    dataset,
    batch_size=10,
    num_workers=1,
    shuffle=False
)


mean = 0.
std = 0.
nb_samples = 0.
for data in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(mean, std)

tensor([-0.0007,  0.0002,  0.0010]) tensor([1.0000, 0.9989, 0.9987])


In [47]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = gdata
        
    def __getitem__(self, index):
        x = self.data[index]
        return x

    def __len__(self):
        return len(self.data)

def online_mean_and_sd(loader):
    """Compute the mean and sd in an online fashion

        Var[x] = E[X^2] - E^2[X]
    """
    cnt = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)

    for data in loader:

        b, c, h, w = data.shape
        nb_pixels = b * h * w
        sum_ = torch.sum(data, dim=[0, 2, 3])
        sum_of_square = torch.sum(data ** 2, dim=[0, 2, 3])
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)

        cnt += nb_pixels

    return fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)

    
dataset = MyDataset()
loader = DataLoader(
    dataset,
    batch_size=10,
    num_workers=1,
    shuffle=False
)

# mean, std = online_mean_and_sd(loader)
# print(mean, std)

tensor([-0.0007,  0.0002,  0.0010]) tensor([1.0005, 0.9993, 0.9992])
